<a href="https://colab.research.google.com/github/micardona96/ML/blob/master/Etapa_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etapa 1 del Proyecto
### Por Miguel Angel Cardona Chamorro

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, StandardScaler
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import make_pipeline


# Diccionario

| Columna     | Tipo        | Descripción                                               |
|-------------|-------------|-----------------------------------------------------------|
| season      | categórica  | Estación del año (Winter, Spring, Summer, Fall)            |
| weekday     | numérico    | Día de la semana (de 1 a 7)                                |
| weathersit  | categórica  | Clima (Clear, Mist, Light Rain, Heavy Rain)                |
| temp        | numérico    | Temperatura                                               |
| atemp       | numérico    | Sensación de temperatura                                   |
| hum         | numérico    | Humedad                                                   |
| windspeed   | numérico    | Velocidad del viento                                      |
| cnt         | numérico    | Cantidad de bicicletas rentadas                           |
| time_of_day | categórica  | Parte del día (Morning, Evening, Night)                   |


In [ ]:
ruta = './data-etapa1.csv'

data_raw =  pd.read_csv(ruta)
data_raw

,season,weekday,weathersit,temp,atemp,hum,windspeed,cnt,time_of_day
0,Winter,6,Clear,3.28,3.0014,0.81,0.0000,16,Night
1,Winter,6,Clear,2.34,1.9982,0.80,0.0000,40,Night
2,Winter,6,Clear,2.34,1.9982,0.80,0.0000,32,Night
3,Winter,6,Clear,3.28,3.0014,0.75,0.0000,13,Night
4,Winter,6,Clear,3.28,3.0014,0.75,0.0000,1,Night
...,...,...,...,...,...,...,...,...,...
17374,Winter,1,Mist,4.22,1.0016,0.60,11.0014,119,Evening
17375,Winter,1,Mist,4.22,1.0016,0.60,11.0014,89,Night
17376,Winter,1,Clear,4.22,1.0016,0.60,11.0014,90,Night
17377,Winter,1,Clear,4.22,1.9982,0.56,8.9981,61,Night


🔥 El conjunto de datos contiene +17k registros y 9 columnas. Es importante destacar que la columna weekday debe ser tratada como un parámetro categórico, ya que los días de la semana representados en formato numérico (de 1 a 7) no deben ser considerados como valores numéricos, sino como índices correspondientes a sus nombres.

# Exploración y perfilamiento de los datos

In [ ]:
data_raw.describe()

,weekday,temp,atemp,hum,windspeed,cnt
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,3.003683,15.358397,15.401157,0.627229,12.736540,189.463088
std,2.005771,9.050138,11.342114,0.192930,8.196795,181.387599
min,0.000000,-7.060000,-16.000000,0.000000,0.000000,1.000000
25%,1.000000,7.980000,5.997800,0.480000,7.001500,40.000000
50%,3.000000,15.500000,15.996800,0.630000,12.998000,142.000000
75%,5.000000,23.020000,24.999200,0.780000,16.997900,281.000000
max,6.000000,39.000000,50.000000,1.000000,56.996900,977.000000


# Limpieza y preparación de los datos

In [ ]:
duplicated = data_raw.duplicated().sum()
duplicated

42

🔥 Identificamos 42 registros duplicados en el conjunto de datos. En el contexto de este problema, podemos argumentar que estos duplicados son resultado de errores en la captura de datos, ya que cada registro debería representar una instancia única y no debería haber duplicados.

In [ ]:
data_raw.isna().sum()

,0
season,0
weekday,0
weathersit,0
temp,0
atemp,0
hum,0
windspeed,0
cnt,0
time_of_day,0


🔥 No obtenemos valores null

In [ ]:
data = data_raw.copy()
data = pd.get_dummies(data, columns=[ 'weekday', 'season', 'weathersit', 'time_of_day'], drop_first=True)
data

,temp,atemp,hum,windspeed,cnt,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,season_Spring,season_Summer,season_Winter,weathersit_Heavy Rain,weathersit_Light Rain,weathersit_Mist,time_of_day_Morning,time_of_day_Night
0,3.28,3.0014,0.81,0.0000,16,False,False,False,False,False,True,False,False,True,False,False,False,False,True
1,2.34,1.9982,0.80,0.0000,40,False,False,False,False,False,True,False,False,True,False,False,False,False,True
2,2.34,1.9982,0.80,0.0000,32,False,False,False,False,False,True,False,False,True,False,False,False,False,True
3,3.28,3.0014,0.75,0.0000,13,False,False,False,False,False,True,False,False,True,False,False,False,False,True
4,3.28,3.0014,0.75,0.0000,1,False,False,False,False,False,True,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,4.22,1.0016,0.60,11.0014,119,True,False,False,False,False,False,False,False,True,False,False,True,False,False
17375,4.22,1.0016,0.60,11.0014,89,True,False,False,False,False,False,False,False,True,False,False,True,False,True
17376,4.22,1.0016,0.60,11.0014,90,True,False,False,False,False,False,False,False,True,False,False,False,False,True
17377,4.22,1.9982,0.56,8.9981,61,True,False,False,False,False,False,False,False,True,False,False,False,False,True


🔥 Convertimos las variables categóricas en variables numéricas mediante la técnica de One-Hot Encoding, entre ellas weekday. ahora obtenemos 19 columnas.

# División del conjunto de datos en entrenamiento y prueba

In [ ]:
X = data.drop('cnt', axis=1)
Y = data['cnt']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=77)

kfold = KFold(n_splits=10, shuffle=True, random_state=77)

print(f"X_train shape: {X_train.shape}")
print(f"Y_train shape: {Y_train.shape}\n")

print(f"X_test shape: {X_test.shape}")

print(f"Y_test shape: {Y_test.shape}")

X_train shape: (13903, 18)
Y_train shape: (13903,)

X_test shape: (3476, 18)
Y_test shape: (3476,)


🔥 En esta sección, construimos dos conjuntos: uno para pruebas y otro para entrenamiento. Ambos conjuntos se utilizarán tanto en el modelo Lasso como en el modelo Polinomial.

# Modelo de regresión Polinomial

In [ ]:
X_train_poly = X_train.copy()
Y_train_poly = Y_train.copy()

pipeline_poly = make_pipeline(
    PolynomialFeatures(),
    RobustScaler(),
    LinearRegression()
)

param_grid_poly = {
    'polynomialfeatures__degree': [2, 3]
}


grid_search_poly = GridSearchCV(pipeline_poly, param_grid_poly, cv=kfold, scoring='neg_root_mean_squared_error', n_jobs=1)
grid_search_poly.fit(X_train_poly, Y_train_poly)


mejor_modelo_poly = grid_search_poly.best_estimator_
mejor_degree_poly = grid_search_poly.best_params_
mejor_rmse_poly = grid_search_poly.best_score_

print(f"Mejor grado de polinomio: {mejor_degree_poly}")
print(f"RMSE del mejor modelo polinomial: {mejor_rmse_poly}")



Mejor grado de polinomio: {'polynomialfeatures__degree': 3}
RMSE del mejor modelo polinomial: -2853736596.768258


🔥 Entrenando el modelo de regresión lineal, donde primero se crean copias de los datos de entrenamiento, luego se construye un pipeline que incluye la expansión de características polinomiales, escalado robusto y al final la regresión lineal.
A continuación, se usa GridSearchCV para buscar el mejor grado del polinomio (entre 2 y 3) utilizando validación cruzada, optimizando el RMSE.

# Regresión regularizada Lasso

In [ ]:
X_train_lasso = X_train.copy()
Y_train_lasso = Y_train.copy()

pipeline_lasso = make_pipeline(
    StandardScaler(),
    Lasso()
)

param_grid_lasso = {
    'lasso__alpha': [1, 2, 3, 4, 5]
}

grid_lasso = GridSearchCV(pipeline_lasso, param_grid_lasso, cv=kfold, scoring='neg_root_mean_squared_error', n_jobs=1)
grid_lasso.fit(X_train_lasso, Y_train_lasso)

mejor_modelo_lasso = grid_lasso.best_estimator_
mejor_alpha_lasso = grid_lasso.best_params_['lasso__alpha']
mejor_rmse_lasso = grid_lasso.best_score_

print(f"Mejor valor de alpha: {mejor_alpha_lasso}")
print(f"RMSE del mejor modelo Lasso: {mejor_rmse_lasso}")


Mejor valor de alpha: 1
RMSE del mejor modelo Lasso: -137.00966548675868


🔥 Entrenando el modelo de regresión Lasso utilizando un pipeline que primero estandariza las características y luego aplica el modelo Lasso. Utiliza GridSearchCV para encontrar el mejor valor de alpha entre los valores 1 a 5, empleando validación cruzada optimizando el RMSE.

In [ ]:
coeficientes = abs(mejor_modelo_lasso.named_steps['lasso'].coef_)

importancia_variables = pd.DataFrame({
    'Variable': X_train.columns,
    'Importancia': coeficientes
})


importancia_variables = importancia_variables.sort_values(by='Importancia', ascending=False)

importancia_variables.reset_index(drop=True, inplace=True)

importancia_variables

,Variable,Importancia
0,time_of_day_Night,89.302117
1,atemp,39.372680
2,time_of_day_Morning,38.855075
3,hum,29.604704
4,season_Winter,23.802170
5,season_Summer,19.219517
6,temp,19.094378
7,weathersit_Light Rain,10.531905
8,season_Spring,10.468935
9,weekday_6,1.351229


🔥 Aqui vemos la importancia de cada variable en el modelo de regresión Lasso. Primero, obtiene los coeficientes absolutos del mejor modelo. Luego, creamos un DataFrame que asocia cada variable con su coeficiente correspondiente. y al final, ordena estas variables de mayor a menor importancia.

# Comparativa

In [ ]:
Y_pred_poly = mejor_modelo_poly.predict(X_test.copy())
rmse_poly_test = np.sqrt(mean_squared_error(Y_test, Y_pred_poly))
r2_poly_test = r2_score(Y_test, Y_pred_poly)
mae_poly_test = mean_absolute_error(Y_test, Y_pred_poly)

Y_pred_lasso = mejor_modelo_lasso.predict(X_test.copy())
rmse_lasso_test = np.sqrt(mean_squared_error(Y_test, Y_pred_lasso))
r2_lasso_test = r2_score(Y_test, Y_pred_lasso)
mae_lasso_test = mean_absolute_error(Y_test, Y_pred_lasso)


results_comparison = pd.DataFrame({
    'Model': ['Poly', 'Lasso'],
    'R2': [r2_poly_test, r2_lasso_test],
    'RMSE': [rmse_poly_test, rmse_lasso_test],
    'MAE': [mae_poly_test, mae_lasso_test]
})

results_comparison

,Model,R2,RMSE,MAE
0,Poly,0.490509,128.505092,93.852608
1,Lasso,0.423238,136.725720,101.875955


🔥 Comparando el rendimiento de los dos modelos entrenados, utilizando un conjunto de datos de prueba. Primero, hace predicciones con cada modelo y calcula tres métricas de evaluación: RMSE , R² y MAE. Luego, organizamos estos resultados en un DataFrame, que muestra las métricas de rendimiento para ambos modelos, permitiendo así una comparación directa de su desempeño en el conjunto de prueba.

# Análisis de resultados.


1. ¿Cuál es el grado de la transformación polinomial que fue seleccionado utilizando la técnica de validación?

    🔥 Grado = 3. Este grado fue elegido por `GridSearchCV` que minimizo el RMSE.






2. ¿Cuál fue el valor de ALFA que fue seleccionado utilizando la técnica de validación para la regresión Lasso?

    🔥 ALFA = 1. Este valor fue elegido por `GridSearchCV` que minimizo el RMSE.




3. A partir de la tabla comparativa, ¿cuál modelo ofrece el mejor rendimiento sobre el conjunto test? ¿Qué interpretación puedes darles a los valores obtenidos sobre las métricas de rendimiento?

    🔥 Con la tabla de comparacion, se puede observar que el modelo de regresion Polinomial ofrece un mejor rendimiento sobre  el conjunto de test en comparación con el modelo Lasso, con un `RMSE` menor (128 frente a 132) y un `MAE` menor (93 frente a 97); Pero el R2 es bajo, lo que indica que aunque el modelo polinomial se ajusta mejor  a los datos,su capacidad para explicar la variabilidad total en la demanda de bicicletas es limitada. entonces los dos modelos tiene un problemas para generalizar correctamente.



4. ¿Cuáles variables fueron seleccionadas con el modelo Lasso? A partir de estas, ¿qué interpretación de cara al problema puedes dar? Reflexiona sobre cómo este nuevo conocimiento podría ayudar a tomar decisiones en el contexto del problema.

    🔥 Las variables más importantes, como `time_of_day_Night`, `atemp`, y `time_of_day_Morning`, indican que la demanda de bicicletas está fuertemente influenciada por la hora del dia, con la noche siendo un factor clave. Esto sugiere que el uso del sistema de alquiler de bicicletas es significativamente menor durante la noche y que la percepción de la temperatura (`atemp`) es más relevante que la temperatura real (`temp`).